In [1]:
using Pkg

Pkg.add("CuArrays")

Pkg.add("CUDAnative")

Pkg.add("CUDAdrv")

using CUDAnative, CUDAdrv, CuArrays

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


Internal error: encountered unexpected error in runtime:
BoundsError(a=Array{Core.Compiler.BasicBlock, (32,)}[
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=1, last=7), preds=Array{Int64, (1,)}[32], succs=Array{Int64, (1,)}[2]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=8, last=14), preds=Array{Int64, (1,)}[1], succs=Array{Int64, (2,)}[5, 3]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=15, last=20), preds=Array{Int64, (1,)}[2], succs=Array{Int64, (1,)}[4]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=21, last=21), preds=Array{Int64, (1,)}[3], succs=Array{Int64, (1,)}[7]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=22, last=28), preds=Array{Int64, (1,)}[2], succs=Array{Int64, (1,)}[6]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=29, last=29), preds=Array{Int64, (1,)}[5], succs=Array{Int64, (1,)}[7]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=30, last=

In [2]:
Pkg.add("BenchmarkTools")

Pkg.add("Test")

using BenchmarkTools, Test

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [24]:
#Initializing necessary matrices and variables

N=parse(Int64,readline())

M1=rand(Float32,N,N)

M2=rand(Float32,N,N)

ParallelM=zeros(Float32,N,N)

SerialM=zeros(Float32,N,N);


stdin> 200


In [27]:
#Naive Matrix Multiplication in the Sequential format

function sequential_multiplication!(M1,M2,SerialM,N)

 for i in 1:N

  for j in 1:N

  sum=0

   for k in 1:N

    sum+=M1[CartesianIndex(i,k)]*M2[CartesianIndex(k,j)]

   end

   SerialM[CartesianIndex(i,j)]=sum

  end

 end

end 

sequential_multiplication! (generic function with 1 method)

In [28]:
#Transferring data to the device from the host

D_ParallelM=CuArray(ParallelM)

D_M1=CuArray(M1)

D_M2=CuArray(M2)


200×200 CuArray{Float32,2}:
 0.527656   0.710957   0.338796   …  0.189387   0.710702   0.973829   
 0.32401    0.347381   0.518803      0.308182   0.969847   0.485042   
 0.57957    0.759278   0.680202      0.245455   0.363693   0.0869473  
 0.716606   0.475484   0.0332795     0.753304   0.404614   0.260337   
 0.591824   0.335478   0.712172      0.297172   0.858222   0.000423908
 0.215169   0.373706   0.836659   …  0.884782   0.347741   0.610559   
 0.47026    0.47903    0.242297      0.809379   0.995136   0.130399   
 0.878547   0.296098   0.276701      0.661493   0.5801     0.73938    
 0.516205   0.802405   0.435897      0.465838   0.988598   0.452164   
 0.778921   0.395004   0.400467      0.849023   0.663865   0.33032    
 0.810615   0.168381   0.420969   …  0.656831   0.740862   0.129437   
 0.170093   0.211346   0.924168      0.122106   0.894114   0.322706   
 0.791759   0.434697   0.457093      0.566284   0.0733985  0.347932   
 ⋮                                ⋱              

In [31]:
#Naive Matrix Multiplication in the Parallel Format

function parallel_multiplication!(M1,M2,ParallelM,N)
 sum = 0
 threadId = (blockIdx().x - 1)*blockDim().x*blockDim().y + (threadIdx().y -1)*blockDim().x + threadIdx().x
 x=trunc(Int32,threadId/N) + 1
 y=trunc(Int32,threadId%N) + 1
 for k in 1:N 
  @inbounds sum = sum + M1[CartesianIndex(x,k)] * M2[CartesianIndex(k,y)]
 end
 @inbounds ParallelM[CartesianIndex(x,y)] = sum
 return
end


parallel_multiplication! (generic function with 1 method)

In [32]:
@btime sequential_multiplication!(M1,M2,SerialM,N)
@btime @cuda blocks = trunc(Int32,(N^2)/1024) threads = (32,32) parallel_multiplication!(D_M1,D_M2,D_ParallelM,N)
H_ParallelM=Array(D_ParallelM)
@test isapprox(SerialM,H_ParallelM)

  11.150 ms (0 allocations: 0 bytes)
  10.158 μs (61 allocations: 2.33 KiB)


Test Passed